In [14]:
import pandas as pd
import numpy as np
import os


In [15]:
def namefinder(nlist,which):
    for name in nlist:
        if which in name:
            return name

In [16]:
def getfiles(start="bhist",items='120',mis=0):
    listoffiles=[]
    where='../DataDump'
    for file in os.listdir(where):
        if file.startswith(start):
            if items in file:
                if mis==0:
                    if '50' not in file:
                        listoffiles.append(file)
                elif mis==1:
                    if '50' in file:
                        listoffiles.append(file)
                else:
                    listoffiles.append(file)
    return where,listoffiles

In [17]:
def getthemean(filename,colver='iteration',whichcol=['hr_top10']):
    data=pd.read_csv(filename)
    num=int(np.max(data[colver]))
    meandata=data[data[colver]==1][whichcol].get_values()
    for i in range(2,num+1):
        meandata+=data[data[colver]==i][whichcol].get_values()
    meandata/=num
    return meandata

In [18]:
def createlabel(f):
    name=f.split()
    label=[]
    typeoflab=name[1][1:]
    label.append(typeoflab)
    if typeoflab=='TS' or typeoflab=='greedy':
        label.append(' e=')
        label.append(str(int(name[14])/int(name[6]))[:5])
        label.append(' d=')
        label.append(name[15][3:])
    if typeoflab=='learnthenearn':
        label.append(' a=')
        label.append(name[-2])
        if name[-5]!='0':
            label.append(' misinf')
    elif name[-2]!='0':
        label.append(' misinf')
    return ''.join(label)

In [19]:
def firstable():
    items='120'
    mis=-1
    where,listoffiles=getfiles(start="hrhist",items=items,mis=mis)
    whichcol=['hr_top3','hr_top10']
    data={}
    for i,f in enumerate(listoffiles):
        meandata=getthemean(where+'/'+f,colver='iteration',whichcol=whichcol)*100
        label=createlabel(f)
        data[label]=[meandata[12,0],meandata[50,0],meandata[12,1],meandata[50,1]]
    index=['top3_resp260','top3_resp1020','top10_resp260','top10_resp1020']
    table= pd.DataFrame(data, index=index)
    table.to_csv('tables/Explore_params.csv', float_format='%.2f')

In [58]:
def efficiencyGain(table,which):
    data={}
    mask=table[which]>90.
    if mask.any():
        base=int((table[which][mask]).index[0])
    else:
        base=1020
    for name in table.columns:
        mask=table[name]>table[which]['1020']
        if mask.any():
            loc=(table[name][mask]).index[0]
            data[name]=[float(loc),float(1020/int(loc))]
        else:
            data[name]=[float(0),float(0)]
        mask=table[name]>90.
        if mask.any():
            loc=(table[name][mask]).index[0]
            data[name]+=[float(loc),float(base/int(loc))]
        else:
            data[name]+=[float(0),float(0)]
    index=['Reach at (end)','Efficiency Gain (end)','Reach at (.9)','Efficiency Gain (.9)']
    table= pd.DataFrame(data, index=index)
    return table

In [62]:
def createtables(items='120',mis=0,whichcol=['hr_top3']):
    index=[str(x) for x in range(20,1040,20)]
    where,listoffiles=getfiles(start="hrhist",items=items,mis=mis)

    data={}
    for i,f in enumerate(listoffiles):
        meandata=getthemean(where+'/'+f,colver='iteration',whichcol=whichcol)*100
        label=createlabel(f)
        data[label]=meandata[:,0]
    table=pd.DataFrame(data, index=index)
    newtable=efficiencyGain(table,namefinder(table.columns,'fixed'))
    thefinaltable=table.iloc[::5].append(newtable)
    lab=items+'itemstop'+whichcol[0][6:]
    if mis==1:
        lab+='misinf'
    elif mis==-1:
        lab+='all'
    thefinaltable.to_csv('tables/Method_Summary'+lab+'.csv', float_format='%.2f')

In [63]:
def secondtable():
    for items in ['40','120','300']:
        for mis in [0,1]:
            for col in ['hr_top3','hr_top10']:
                createtables(items=items,mis=mis,whichcol=[col])

In [64]:
firstable()
secondtable()